# Plot individual images on the cluster

This notebook explores how to view geospatial data on the cluster without needing to transfer it

## Imports

In [ ]:
import os
import sys
from IPython.display import Image

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from FileInfo import *
from FileChecks import *
from plot import *

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basicConfig
print("Basic Parameters: \n PrintDate = {} \n brdf_dir = {} \n data source = {} \n gridCell = {} \n index_dir = {} \n out_dir = {} \n plotYr = {}"
      .format(basicConfig['today'],basicConfig['brdf_dir'],basicConfig['data_source'],basicConfig['gridCell'],basicConfig['index_dir'],basicConfig['out_dir'], basicConfig['plotYr']))

%store -r SinglePlotParams
print("Plotting Parameters: \n plotDay = {} \n Viewband = {} \n imageType = {}"
      .format(SinglePlotParams['plotDay'],SinglePlotParams['Viewband'],SinglePlotParams['imageType']))

In [ ]:
print('closest downloaded Sentinel image is:')
SampImg_raw_Sentinel = GetClosestImage(os.path.join(basicConfig['raw_dir'],'{:06d}'.format(basicConfig['gridCell'])), 'Sentinel', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest Sentinel brdf is:')
SampImg_brdf_Sentinel = GetClosestImage(basicConfig['brdf_dir'], 'Sentinel', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest downloaded Landsat image is:')
SampImg_raw_Landsat = GetClosestImage(os.path.join(basicConfig['raw_dir'],'{:06d}'.format(basicConfig['gridCell'])), 'Landsat', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest downloaded Landsat5 image is:')
SampImg_raw_Landsat = GetClosestImage(os.path.join(basicConfig['raw_dir'],'{:06d}'.format(basicConfig['gridCell'])), 'Landsat5', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest downloaded Landsat7 image is:')
SampImg_raw_Landsat = GetClosestImage(os.path.join(basicConfig['raw_dir'],'{:06d}'.format(basicConfig['gridCell'])), 'Landsat7', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest downloaded Landsat8 image is:')
SampImg_raw_Landsat = GetClosestImage(os.path.join(basicConfig['raw_dir'],'{:06d}'.format(basicConfig['gridCell'])), 'Landsat8', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest downloaded Landsat9 image is:')
SampImg_raw_Landsat = GetClosestImage(os.path.join(basicConfig['raw_dir'],'{:06d}'.format(basicConfig['gridCell'])), 'Landsat9', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest Landsat brdf is:')
SampImg_brdf_Landsat = GetClosestImage(basicConfig['brdf_dir'], 'Landsat', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
print('closest smoothed {} index image is:'.format(basicConfig['spec_index']))
SampImg_Smooth = GetClosestImage(basicConfig['index_dir'], 'Smooth', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])

In [ ]:
if SinglePlotParams['imageType'] == 'Smooth':
    #GetValidPixPer(SampImg_Smooth)
    exploreBand(SampImg_Smooth, basicConfig['spec_index'])
elif SinglePlotParams['imageType'] in ['Sentinel','AllRaw']:
    #GetValidPixPer(SampImg_brdf_Sentinel)
    exploreBand(SampImg_brdf_Sentinel, SinglePlotParams['Viewband'])
elif SinglePlotParams['imageType'] == 'Landsat':
    #GetValidPixPer(SampImg_brdf_Landsat)
    exploreBand(SampImg_brdf_Landsat, SinglePlotParams['Viewband'])

In [ ]:
exploreBand(SampImg_raw_Sentinel, SinglePlotParams['Viewband']);

In [ ]:
exploreBand(SampImg_raw_Landsat, SinglePlotParams['Viewband']);

## Plot 3-band image

In [ ]:
fig, axarr = plt.subplots(1, 2, figsize=(15,5))
s_rgb = GetrbgImg(SampImg_raw_Sentinel,SinglePlotParams['gamma'])
l_rgb = GetrbgImg(SampImg_raw_Landsat,SinglePlotParams['gamma'])
#fig = plt.figure(figsize=(18,12))
axarr[0].imshow(l_rgb)
axarr[0].set_title("Landsat")
axarr[0].axis('off')
axarr[1].imshow(s_rgb)
axarr[1].set_title("Sentinel")
axarr[1].axis('off');

In [ ]:
l_rgb = GetrbgImg(SampImg_Landsat,SinglePlotParams['gamma'])
fig = plt.figure(figsize=(18,12))
plt.imshow(l_rgb)
plt.title("Landsat")
plt.axis('off');

## Compare original, downloaded and brdf image:

In [ ]:
l_orig_id = os.path.basename(SampImg_raw_Landsat).strip('.tif')
l_orig = GetImgFromPlanetaryHub(l_orig_id)
Image(url=l_orig.assets["rendered_preview"].href, width=500)

In [ ]:
fig, axarr = plt.subplots(2, 2, figsize=(10,10))
l_dl = GetrbgImg(SampImg_raw_Landsat,1)
l_brdf = GetrbgImg(SampImg_brdf_Landsat,SinglePlotParams['gamma'])
s_dl = GetrbgImg(SampImg_raw_Sentinel,SinglePlotParams['gamma'])
s_brdf = GetrbgImg(SampImg_brdf_Sentinel,SinglePlotParams['gamma'])

axarr[0,0].imshow(l_dl)
axarr[0,0].set_title("original Landsat download")
axarr[0,0].axis('off')
axarr[0,1].imshow(l_brdf)
axarr[0,1].set_title("Landsat brdf")
axarr[0,1].axis('off')
axarr[1,0].imshow(s_dl)
axarr[1,0].set_title("original Sentinel download")
axarr[1,0].axis('off')
axarr[1,1].imshow(s_brdf)
axarr[1,1].set_title("Sentinel brdf")
axarr[1,1].axis('off');

In [ ]:
### Make valid pixel db for brdf and orig images
##  NOTE: THis can be heavy and best run through Slurm with the .sh script and then loaded as below
#validPixdb = checkValidPixels(basicConfig['raw_dir'],basicConfig['brdf_dir'],basicConfig['gridCell'],'Landsat8',Yrs=[2021,2021],dataSource='stac')

Processing_db = ReadDB(os.path.join(basicConfig['raw_dir'],'{:06d}'.format(basicConfig['gridCell']),'processed_imgs_All.info'))
Processing_db.head(n=5)

In [ ]:
dfslice = Processing_db['{}-01-01'.format(basicConfig['plotYr']):'{}-12-31'.format(basicConfig['plotYr'])]
brdf_file_list = dfslice['file_path'].tolist()
dl_file_list = dfslice['orig_file_path'].tolist()
#print('There are {} files in file list'.format(len(file_list)))
time_names = dfslice.date.dt.strftime('%Y-%m-%d').values.tolist()
validpix_brdf = getNumValidPix_forStac(brdf_file_list)
validpix_orig = getNumValidPix_forStac(dl_file_list, date_list=time_names)

fig, axarr = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('Num valid pixels for {}'.format(basicConfig['plotYr']))
rawim = axarr[0].imshow(validpix_orig, cmap='jet_r')
axarr[0].set_title("original downloads")
bimg = axarr[1].imshow(validpix_brdf, cmap='jet_r')
axarr[1].set_title("brdfs")
fig.colorbar(rawim, ax=axarr[0], fraction=0.046, pad=0.04) 
fig.colorbar(bimg, ax=axarr[1], fraction=0.046, pad=0.04) ;

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

outName = str(basicConfig['country']+'1b_SampleImagePlot_for_Cell'+str(basicConfig['gridCell'])+'_from_'+str(SinglePlotParams['plotYr'])+str(SinglePlotParams['plotDay']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1b_ExploreData_OpenImage.ipynb